# Обучение детектора YOLOv5, конвертация в ONNX. 

In [1]:
'''!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install'''

import xml.etree.ElementTree as ET
from pathlib import Path
import glob
import shutil
import os
import json
from typing import Union
from PIL import Image, ImageDraw
import numpy as np
from sklearn.model_selection import train_test_split

from openvino.runtime import Core

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-303-g2430578 Python-3.9.7 torch-1.10.0+cu113 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6075MiB)


Setup complete ✅ (6 CPUs, 31.3 GB RAM, 167.8/233.2 GB disk)


Вспомогательные функции

In [2]:
def create_folder(*path):
    if not os.path.exists(os.path.join(*path)):
        os.mkdir(os.path.join(*path))
        
def put_down_files(files: list, folder: str) -> None:
    for file in files: 
        dest_path = os.path.join(file.parent, folder, file.name)
        os.rename(file, dest_path)

def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]


Типы файлов изображений, пути к каталогам, размер изображения:

In [3]:
file_extencions = ["jpg", "jpeg", "png", "webp"]
input_dir = "/media/maksim/Ubuntu 20.0/Cocktails/"
labels_dir = "/home/maksim/gitrepo/datasets/cocktails/labels/"
image_dir = "/home/maksim/gitrepo/datasets/cocktails/images/"
IMAGE_SIZE = 640

### Конвертация датасета в формат YOLO, изменение размера изображений.

In [ ]:
# create the labels folder (output directory)
create_folder(labels_dir)
create_folder(image_dir)

# identify all the xml files in the annotations folder (input directory)
files = sorted(list(Path(input_dir).rglob('*.xml')))
files = [str(path) for path in files]
classes = []
file_num = 0

for file in files:
    
    filename = os.path.splitext(os.path.basename(file))[0]
    perent_dir = (os.path.split(file)[0]).split('/')[-1]
    # check if the label contains the corresponding image file
    
    image_file = None
    for ext in file_extencions:
        if os.path.exists(os.path.join(input_dir, perent_dir, f"{filename}.{ext}")):
            image_file = ".".join(file.split(".")[:-1] + [ext])
            break
    
    if image_file is None:
        print(f"{os.path.join(input_dir, perent_dir)}/{filename} not exist!")
        continue
    
    try:
        image = Image.open(image_file).convert("RGB")
    except Exception as ex:
        print(ex)
        continue
        
    file_num += 1
    new_path = os.path.join(image_dir, f"{file_num:08d}.jpg")
    image.resize((IMAGE_SIZE, IMAGE_SIZE)).save(new_path, "JPEG")
        
    result = []

    # parse the content of the xml file
    tree = ET.parse(file)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find("name").text
        # check for new classes and append to list
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(dim.text) for dim in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    with open(os.path.join(labels_dir, f"{file_num:08d}.txt"), "w", encoding="utf-8") as f:
        f.write("\n".join(result))

### Разбивка датасета на трейн и валидацию, выполняется при необходимости:

In [6]:
# Trait/validation split
files = sorted(list(Path(image_dir).rglob('*.*')))
labels = sorted(list(Path(labels_dir).rglob('*.*')))
full_files = [path.name[:-4] for path in files]
full_labels = [path.name[:-4] for path in labels]

train_files, val_files, train_labels, val_labels = train_test_split(files, labels, test_size=960, shuffle=True)

# create the labels folder (output directory)
create_folder(labels_dir, "train")
create_folder(labels_dir, "val")
create_folder(image_dir, "train")
create_folder(image_dir, "val")
    
put_down_files(train_files, "train")
put_down_files(val_files, "val")
put_down_files(train_labels, "train")
put_down_files(val_labels, "val")

In [ ]:
# Tensorboard (optional)
%load_ext tensorboard
%tensorboard --logdir runs/train

In [5]:
# Weights & Biases (optional)
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: polushinm. Use `wandb login --relogin` to force relogin


True

## Обучение YOLO

In [6]:
# Train YOLOv5s
!python train.py --img {IMAGE_SIZE} --batch 48 --epochs 180 --data cocktails.yaml --weights yolov5n.pt --cache

wandb: Currently logged in as: polushinm. Use `wandb login --relogin` to force relogin
train: weights=yolov5n.pt, cfg=, data=cocktails.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=180, batch_size=48, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 3 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-303-g2430578 Python-3.9.7 torch-1.10.0+cu113 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6075MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0


     Epoch   gpu_mem       box       obj       cls    labels  img_size
     6/179     4.67G   0.03277   0.01659         0        43       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        100        116      0.802      0.836      0.867      0.493

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     7/179     4.67G   0.03016   0.01574         0        40       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        100        116      0.874      0.707      0.835      0.533

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     8/179     4.67G    0.0305   0.01571         0        48       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        100        116      0.926      0.869      0.936      0.622

     Epoch   gpu_mem       box      


     Epoch   gpu_mem       box       obj       cls    labels  img_size
    57/179     4.67G   0.02255   0.01289         0        44       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        100        116      0.958      0.985      0.991      0.813

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    58/179     4.67G   0.02301   0.01289         0        40       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        100        116      0.991      0.936      0.988      0.831

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    59/179     4.67G   0.02287   0.01293         0        53       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        100        116      0.966      0.983      0.991      0.832

     Epoch   gpu_mem       box      


     Epoch   gpu_mem       box       obj       cls    labels  img_size
   108/179     4.67G   0.02026   0.01161         0        46       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        100        116      0.991      0.991      0.995      0.889

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   109/179     4.67G   0.02045   0.01168         0        39       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        100        116      0.991      0.991      0.995      0.884

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   110/179     4.67G   0.02019   0.01164         0        57       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        100        116      0.989      0.991      0.995      0.888

     Epoch   gpu_mem       box      


     Epoch   gpu_mem       box       obj       cls    labels  img_size
   159/179     4.67G   0.01776   0.01074         0        41       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        100        116       0.99      0.991      0.995      0.925

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   160/179     4.67G   0.01815   0.01077         0        38       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        100        116       0.99      0.991      0.995      0.926

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   161/179     4.67G   0.01728   0.01061         0        45       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        100        116       0.99      0.991      0.995      0.926

     Epoch   gpu_mem       box      

### Экспорт модели в формат ONNX

In [14]:
!python export.py --img 640 --weights best.pt --include onnx

export: data=data/coco128.yaml, weights=['best.pt'], imgsz=[640], batch_size=1, device=cpu, half=False, inplace=False, train=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v6.1-303-g2430578 Python-3.9.7 torch-1.10.0+cu113 CPU

Fusing layers... 
[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.
Model summary: 213 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs

PyTorch: starting from best.pt with output shape (1, 25200, 6) (3.7 MB)

ONNX: starting export with onnx 1.11.0...
ONNX: export success, saved as best.onnx (7.1 MB)

Export complete (8.60s)
Results saved to /home/maksim/gitrepo/yolov5
Detect:          python detect.py --weights best.onnx 
Validate:        python val.py --weights best.onnx 
PyTorch Hub:     model = torch.hub.load('ultralytics/yolov5', 'custom'

### Проверка инференса ONNX через OpenVino

In [15]:
# Opening model JSON config
with open('/home/maksim/Cocktails/config/model_detector.json', 'r') as f:
    model_conf = json.load(f)

IMAGE_SIZE = model_conf['IMAGE_SIZE']
SIZE_QUANT = model_conf['SIZE_QUANT']
BBOX_ECCENTRICITY_PENALTY_POWER = model_conf['BBOX_ECCENTRICITY_PENALTY_POWER']
BBOX_SIZE_PENALTY_POWER = model_conf['BBOX_SIZE_PENALTY_POWER']

In [16]:
open_vino_core = Core()
onnx_detector_path = 'best.onnx'
detector_onnx = open_vino_core.read_model(model=onnx_detector_path)
compiled_detector_onnx = open_vino_core.compile_model(model=detector_onnx, device_name="CPU")

In [17]:
def open_image(path: str, image_size: int, quant: int) -> np.array:
    try:
        image = Image.open(path).convert("RGB")
    except Exception:
        return None
    
    input_width, input_height = image.size
    if input_width >= input_height:
        width = round(image_size / quant) * quant
        height = round(image_size / quant) * quant
    else:
        height = round(image_size / quant) * quant
        width = round(image_size / quant) * quant
        
    return np.asarray(image.resize((width, height))) / 255
  
def predict_bbox(model_onnx, 
            path: str, 
            image_size: int, 
            threshold: float = 0.05, 
            expansion: float = 1., 
            quant: int = 32,
            size_pen_pow = 0.,
            eccentr_pen_pow = 0.,
           ) -> Union[tuple[float], None]:
    
    img = open_image(path, image_size, quant=quant)
    
    if img is not None:
        
        width = img.shape[1]
        height = img.shape[0]
        inputs = [np.moveaxis(img, 2, 0)[None, :, :, :]]
        outputs = model_onnx(inputs)[model_onnx.output(0)][0]
        
        conf_filt = outputs[outputs[:, 4] > threshold]
        confidence = conf_filt[:, 4] * (conf_filt[:, 5]) # TODO: get rid of this kluge
        
        if confidence.max() > threshold:
            
            penalties = (conf_filt[:, 2] * conf_filt[:, 3])**size_pen_pow / \
                    abs((conf_filt[:, 0] - width/2)**2 + (conf_filt[:, 1] - height/2)**2)**eccentr_pen_pow
            confidence_penalted = confidence * penalties
            
            bbox = conf_filt[confidence.argmax(), [0, 1, 2, 3]]
            xmin = (bbox[0] - bbox[2]*0.5*expansion) / width
            ymin = (bbox[1] - bbox[3]*0.5*expansion) / height
            xmax = (bbox[0] + bbox[2]*0.5*expansion) / width
            ymax = (bbox[1] + bbox[3]*0.5*expansion) / height
            
            return xmin, ymin, xmax, ymax
        
    return None


def add_bounding_box(path: str, b_box: tuple[float], thickness: float = 7e-3) -> None:
    xmin, ymin, xmax, ymax = b_box
    with Image.open(path).convert("RGB") as image:
        width, height = image.size
        xmin, xmax = round(xmin*width), round(xmax*width)
        ymin, ymax = round(ymin*height), round(ymax*height)
        
        line_width = round((width + height)*thickness)
        
        draw = ImageDraw.Draw(image)
        draw.line([(xmin, ymin), 
                   (xmax, ymin), 
                   (xmax, ymax), 
                   (xmin, ymax), 
                   (xmin, ymin)], 
                  fill="#51546c", 
                  width=line_width, 
                  joint='curve')
        image.save(path, "JPEG")

In [18]:
im_path = '/home/maksim/Cocktails/Images/Coctails_raw/Apple_martini/image17.jpeg'
new_path = im_path.split("/")[-1]

shutil.copyfile(im_path, new_path)

bbox = predict_bbox(compiled_detector_onnx, #infer_request, 
                    new_path, 
                    image_size=IMAGE_SIZE, 
                    expansion=1.0, 
                    quant=SIZE_QUANT,
                    size_pen_pow = BBOX_SIZE_PENALTY_POWER,
                    eccentr_pen_pow = BBOX_ECCENTRICITY_PENALTY_POWER,
                   )
bbox

(0.24819059371948243,
 0.3640887975692749,
 0.827974796295166,
 0.9910634279251098)

In [19]:
add_bounding_box(new_path, bbox, thickness=7e-3)

In [ ]:
Image.open(new_path)

#### References:

https://github.com/ultralytics/yolov5/blob/master/tutorial.ipynb \
https://towardsdatascience.com/convert-pascal-voc-xml-to-yolo-for-object-detection-f969811ccba5